# reinforcement learning using light sensor

### Setup

In [120]:
import random, heapq, rpyc
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from IPython import display
from mpl_toolkits.mplot3d import Axes3D



conn = rpyc.classic.connect("ev3dev.local") # host name or IP address of the EV3
ev3 = conn.modules['ev3dev2']      # import ev3dev2.ev3 remotely

legosensors=conn.modules['ev3dev2.sensor.lego']
sensor_controls=conn.modules['ev3dev2.sensor']
sound=conn.modules['ev3dev2.sound']
led_controls=conn.modules['ev3dev2.led']

led=led_controls.Leds()
sound=sound.Sound()
        

In [121]:
out1=sensor_controls.INPUT_1
out2=sensor_controls.INPUT_2
out3=sensor_controls.INPUT_3

touch = legosensors.TouchSensor(out3)
touchstop = legosensors.TouchSensor(out2)
color =legosensors.ColorSensor(out1)


In [122]:
def dist(color1,color2):
    col1=list(color1)
    col2=list(color2)
    return(np.sqrt((col1[0]-col2[0])**2+(col1[1]-col2[1])**2+(col1[2]-col2[2])**2))


# KNN simple version
### Data

In [123]:
colorlist=[
    ("Black",(0,0,0)),
    ("Blue",(0,0,255)),
    ("Red",(255,0,0)),
    ("Green",(0,255,0)),
    ("Yellow",(255,255,0)),
    ("White",(255,255,255))
]

In [110]:
while True:
    if touch.is_pressed:
        colorlist.append((color.color_name,color.rgb))
        
    if touchstop.is_pressed:
        break

KeyboardInterrupt: 

In [124]:
def Plot3d(point,dictionary):

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for colortuple in dictionary:

        ax.scatter(colortuple[1][0], colortuple[1][1], colortuple[1][2], c=colortuple[0],)
    
    
    ax.scatter(point[0],point[1],point[2],c="Black",marker="^")
    
    ax.set_xlabel('Red')
    ax.set_ylabel('Green')
    ax.set_zlabel('Blue')

    return(fig)

    

In [125]:
def colorKNN(distlist,k):
    
    dists=[dist[1] for dist in distlist]
    limit=max(heapq.nsmallest(k,dists))
    
    
    kcolors=defaultdict(int)
    for colortuple in distlist:
        if colortuple[1]<=limit:
            kcolors[colortuple[0]]+=1

    maxcolor=max([item for key,item in kcolors.items()])
    returncolors=[]
    for key,item in kcolors.items():
        if item>=maxcolor:
            returncolors.append(key)
            
    return(random.choice(returncolors))
    
    

### Execution

In [113]:
k=2
pred=False
while True:
    if touch.is_pressed and touchstop.is_pressed and pred==False:
        color_raw=color.rgb
                
        distlist=[]
        for colortuple in colorlist:
            distlist.append((colortuple[0],dist(colortuple[1],color_raw)))

        bestcolor=colorKNN(distlist,k)
                
        
        
        
        display.clear_output(wait=True)
        display.display(Plot3d(color_raw,colorlist))

        sound.speak(bestcolor)
        
        pred=True
        
        
    if touchstop.is_pressed and pred==True:
        pred=False
    
    if touch.is_pressed and pred==True:
        pred=False
        colorlist.append((bestcolor,color_raw))
        

KeyboardInterrupt: 

In [15]:
while True:
    display.clear_output(wait=True)
    display.display(color.rgb,color.color_name)
    if touchstop.is_pressed:
        break


(0, 3, 0)

'NoColor'

# Tensorflow neural network

In [126]:
#import keras as ks
#from keras.layers import Input, Dense, Activation
#from keras.models import Model, Sequential
import tensorflow as tf

In [127]:
color_names=["Black","Blue","Green","Red","White","Yellow"]
number_of_colors=len(color_names)
def wrong_color(color):
    return([(1-int(col==color)) for col in color_names])

def correct_color(color):
    return([int(col==color) for col in color_names])
    

In [128]:
tf.reset_default_graph()

lr = 1  # learning rate

# Initialize placeholders
x = tf.placeholder(shape=[None, 3], dtype=tf.float32)
y = tf.placeholder(shape=[None, number_of_colors], dtype=tf.float32)

# Declare model operations
l = tf.layers.dense(x, 10, use_bias=True, activation=tf.nn.relu)

output_layer = tf.layers.dense(l,number_of_colors,use_bias=True,activation=tf.nn.softmax)



# Declare loss function 
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output_layer, labels=y)


# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(lr)
train_step = my_opt.minimize(loss)


with tf.Session() as sess:
    # Initialize variables
    init = tf.global_variables_initializer()
    sess.run(init)
    
    pred=False
    
    while True:
        
        if touch.is_pressed and touchstop.is_pressed and pred==False:
            
            color_raw=[[ele/255 for ele in color.rgb]]
            
            output=sess.run(output_layer,feed_dict={x:color_raw})[0]
            
            
            display.clear_output(wait=True)
            for i in range(number_of_colors):
                display.display(str(color_names[i])+" : "+str(output[i]))
#                display.display(output)
            
            predict=color_names[np.argmax(output)]
            
            sound.speak(predict)
            
            pred=True
            
        if touch.is_pressed and pred==True:
            
            display.display("True")
            
            sess.run(train_step, feed_dict={x: color_raw, y: [correct_color(predict)]})
            
            pred=False
                
        if touchstop.is_pressed and pred==True:
            
            display.display("False")
            
            sess.run(train_step, feed_dict={x: color_raw, y: [wrong_color(predict)]})

            pred=False

'Black : 0.13585351'

'Blue : 0.1406326'

'Green : 0.28741434'

'Red : 0.15058792'

'White : 0.15438895'

'Yellow : 0.13112265'

'False'

KeyboardInterrupt: 